In [39]:
import os
import yaml

import sys
import pandas as pd
import numpy as np
import pickle
import random
from tqdm import tqdm

sys.path.append("pipeline_src/")

from metrics.metrics import Metric
from dataset.dataset import HypernymDataset
from transformers import AutoTokenizer

from dataset.prompt_schemas import (
    hypo_term_hyper,
    predict_child_from_2_parents,
    predict_child_from_parent,
    predict_child_with_parent_and_grandparent,
    predict_children_with_parent_and_brothers,
    predict_parent_from_child_granparent,
    predict_parent_from_child,
)

import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet as wn

seed = 42
random.seed(seed)
np.random.seed(seed)
pd.rand

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [40]:
path_test = '/home/home/v.moskvoretskii/NLP-DL-Project-hypo-to-hyper/babel_datasets/v2_wnet_test.pickle'
path_pred = '/home/home/v.moskvoretskii/NLP-DL-Project-hypo-to-hyper/babel_datasets/_meta-llama-Llama-2-7b-hfwnet2_filtered_instruct_coma_0'

with open(path_pred, 'rb') as f:
    preds = pickle.load(f)

with open(path_test, 'rb') as f:
    test = pickle.load(f)

len(preds), len(test)

In [51]:
from translate import Translator
translator= Translator(from_lang="en",to_lang="ru")

def delete_techniqal(elem, remove=True):
    if isinstance(elem, str):
        if ".n." in elem and remove:
            return elem.split(".")[0].replace("_", " ")
        else:
            return elem.replace("_", " ")

    elif isinstance(elem, list):
        new_words = []
        for word in elem:
            new_words.append(delete_techniqal(word, remove))
        return new_words

if not True:
    res = {'parent': [], 'definition': [], 'children': [], 'translation': []}
    for elem in tqdm(test):
        case = elem["case"]
    # processed_term, target = transforms[case](elem)
        
        if isinstance(elem['parents'], list):
            try:
                definition = wn.synset(elem['parents'][0]).definition()
            except:
                definition = '-'
            parents = ','.join(delete_techniqal(elem['parents']))
        else:
            try:        
                definition = wn.synset(elem['parents']).definition()
            except:
                definition = '-'
            parents = delete_techniqal(elem['parents'])
        

        if isinstance(elem['children'], list):
            children = ','.join(delete_techniqal(elem['children']))
        else:
            children = delete_techniqal(elem['children'])

        translation = translator.translate(children)

        
        
        res['parent'].append(parents)
        res['definition'].append(definition)
        res['children'].append(children)
        res['translation'].append(translation)


    df = pd.DataFrame.from_dict(res)
    df['translation'][df['translation'].str.contains('MYMEMORY')] = ''
    df['label'] = -1
    df.to_csv('all_test_data.csv', index=0)

100%|██████████| 1274/1274 [12:41<00:00,  1.67it/s]


In [109]:
df1 = pd.read_csv('all_test_data.csv')
# intersection_df = df1.sample(200, random_state=seed)
# intersection_df.to_csv('intersection.csv', index=0)

non_intersect = df1.drop(intersection_df.index)
f,s,t = np.array_split(non_intersect, 3)

fi = pd.concat([f, intersection_df])
si = pd.concat([s, intersection_df])
ti = pd.concat([t, intersection_df])

# fi.to_excel('first_part2.xlsx')
# si.to_excel('second_part.xlsx')
# ti.to_excel('third_part.xlsx')

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
